In [ ]:
# Install Required Libraries
!pip install transformers sentence-transformers faiss-cpu gradio pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [1]:
from google.colab import files

# Upload File
uploaded = files.upload()


Saving free_courses_analytics_vidhya.csv to free_courses_analytics_vidhya.csv


In [2]:
import pandas as pd

# Load the uploaded file
df = pd.read_csv('free_courses_analytics_vidhya.csv')

# Display basic info and sample rows
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Course Title  9 non-null      object
 1   Description   9 non-null      object
 2   Curriculum    9 non-null      object
dtypes: object(3)
memory usage: 344.0+ bytes
None
                                        Course Title  \
0           Frameworks for Effective Problem Solving   
1           Anyone can Build AI Agents - Free Course   
2  A Comprehensive Learning Path to Become a Data...   
3  Reimagining GenAI: Common Mistakes and Best Pr...   
4  Coding a ChatGPT-style Language Model from Scr...   

                                         Description   Curriculum  
0  This course provides structured approaches to ...   18 lessons  
1  An introductory course designed to empower ind...    5 lessons  
2  A detailed program outlining the essential ski...  298 lessons  
3  This course delves into the 

In [3]:
import re

# Text Preprocessing
def preprocess(text):
    text = str(text).lower()
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text.strip()

# Apply preprocessing
df['Title_clean'] = df['Course Title'].apply(preprocess)
df['Description_clean'] = df['Description'].apply(preprocess)

# Save cleaned data
df.to_csv('cleaned_courses.csv', index=False)
print("Cleaned dataset saved as 'cleaned_courses.csv'")


Cleaned dataset saved as 'cleaned_courses.csv'


In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load SentenceTransformer Model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate Embeddings
embeddings = model.encode(df['Title_clean'] + ' ' + df['Description_clean'], show_progress_bar=True)

# Save Embeddings
np.save('course_embeddings.npy', embeddings)
print("Embeddings saved as 'course_embeddings.npy'")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Embeddings saved as 'course_embeddings.npy'


In [6]:
!pip install faiss-cpu
import faiss

# Create FAISS Index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Save FAISS Index
faiss.write_index(index, 'courses_index.faiss')
print("FAISS index saved as 'courses_index.faiss'")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 61.8 MB/s eta 0:00:00
FAISS index saved as 'courses_index.faiss'


In [7]:
def search_courses(query, top_k=5):
    query_embedding = model.encode([preprocess(query)])
    distances, indices = index.search(np.array(query_embedding).astype('float32'), top_k)

    results = []
    for idx in indices[0]:
        if idx < len(df):
            results.append({
                'Course Title': df.iloc[idx]['Course Title'],
                'Description': df.iloc[idx]['Description'],

            })
    return results

# Test Search
query = "machine learning"
results = search_courses(query)
for res in results:
    print(res)


{'Course Title': 'Reimagining GenAI: Common Mistakes and Best Practices for Success', 'Description': 'This course delves into the common pitfalls in Generative AI and offers best practices to achieve success in the field.'}
{'Course Title': 'Mastering Multilingual GenAI Open-Weights for Indic Languages', 'Description': 'This course focuses on mastering Generative AI models tailored for Indic languages, emphasizing open-weight models.'}
{'Course Title': 'GenAI Applied to Quantitative Finance: For Control Implementation', 'Description': 'This course applies Generative AI techniques to quantitative finance, focusing on control implementation strategies.'}
{'Course Title': 'Learning Autonomous Driving Behaviors with LLMs & RL', 'Description': 'An advanced course exploring the integration of Large Language Models and Reinforcement Learning in autonomous driving.'}
{'Course Title': 'Navigating LLM Tradeoffs: Techniques for Speed, Cost, Scale & Accuracy', 'Description': 'A course that examine

In [12]:
!pip install gradio
import gradio as gr

def search_interface(query):
    results = search_courses(query)
    if not results:
        return "No relevant courses found. Try another keyword."

    formatted_results = "\n\n".join(
        [f"**{res['Course Title']}**\n{res['Description']}\n[Link]({res.get('Link', '#')})" for res in results]
    )
    return formatted_results

# Launch Gradio Interface
iface = gr.Interface(
    fn=search_interface,
    inputs="text",
    outputs="markdown",
    title="Smart Course Search",
    description="Enter a query to find relevant free courses on Analytics Vidhya."
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://355202bb7ec46edd2f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [13]:
query_embedding = model.encode([preprocess("solving")])
distances, indices = index.search(np.array(query_embedding).astype('float32'), 5)

print("Distances:", distances)
print("Indices:", indices)


Distances: [[1.4496169 1.8257225 1.8651056 1.8797674 1.8815625]]
Indices: [[0 6 3 1 2]]


In [11]:
query = "solving"
results = search_courses(query)
for res in results:
    print(res)


{'Course Title': 'Frameworks for Effective Problem Solving', 'Description': 'This course provides structured approaches to problem-solving, equipping learners with frameworks to tackle complex challenges effectively.'}
{'Course Title': 'Learning Autonomous Driving Behaviors with LLMs & RL', 'Description': 'An advanced course exploring the integration of Large Language Models and Reinforcement Learning in autonomous driving.'}
{'Course Title': 'Reimagining GenAI: Common Mistakes and Best Practices for Success', 'Description': 'This course delves into the common pitfalls in Generative AI and offers best practices to achieve success in the field.'}
{'Course Title': 'Anyone can Build AI Agents - Free Course', 'Description': 'An introductory course designed to empower individuals to create AI agents, regardless of their prior experience.'}
{'Course Title': 'A Comprehensive Learning Path to Become a Data Analyst in 2025', 'Description': 'A detailed program outlining the essential skills and 

In [ ]:
from google.colab import files

files.download('cleaned_courses.csv')
files.download('course_embeddings.npy')
files.download('courses_index.faiss')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>